In [1]:
import sys
sys.path.append('../src')

import requests

import uschartbook.config

from uschartbook.config import *
from uschartbook.utils import *

In [5]:
cols = ['PEEDUCA', 'EDUC', 'BASICWGT', 'AGE', 'MONTH', 'YEAR', 'LFS', 'HRSACTT', 'FEMALE']

educdt = lambda x: np.where((x.EDUC == 'SC') & (x.PEEDUCA == 40), 'SCND', 
                   np.where((x.EDUC == 'SC') & (x.PEEDUCA == 41), 'VOC',
                   np.where((x.EDUC == 'SC') & (x.PEEDUCA == 42), 'AAD', x.EDUC)))

df = ((pd.read_feather(cps_dir/'cps2019.ft', columns=cols).query('AGE > 24 and MONTH > 1')
         .append(pd.read_feather(cps_dir/'cps2020.ft', columns=cols).query('AGE > 24')))
         .assign(EDUCDT = educdt))
df2 = pd.read_feather(cps_dir/'cps2000.ft', columns=cols).query('AGE > 24').assign(EDUCDT = educdt)

df3 = df.query('LFS == "Employed"')
df4 = df2.query('LFS == "Employed"')

In [6]:
data = pd.DataFrame()

In [7]:
data['2000'] = df2.groupby('EDUCDT').BASICWGT.sum() / df2.BASICWGT.sum() * 100
data['latest'] = df.groupby('EDUCDT').BASICWGT.sum() / df.BASICWGT.sum() * 100

data['2000_emp'] = df4.groupby('EDUCDT').BASICWGT.sum() / df4.BASICWGT.sum() * 100
data['latest_emp'] = df3.groupby('EDUCDT').BASICWGT.sum() / df3.BASICWGT.sum() * 100

In [8]:
data = data.loc[['LTHS', 'HS', 'SCND', 'VOC', 'AAD', 'COLL', 'ADV']]

In [9]:
data.index = ['No High School Diploma', 'High School Diploma', 'Some College, No Degree', 'Associate Degree,\\\*Vocational',
              'Associate Degree,\\\*Academic', "Bachelor's Degree", 'Advanced Degree']

In [10]:
data.to_csv(data_dir / 'cps_educ.csv', sep=';', index_label='name')

In [11]:
data

,2000,latest,2000_emp,latest_emp
No High School Diploma,16.080801,9.702842,10.038269,6.786475
High School Diploma,32.861298,28.176260,31.296618,25.174486
"Some College, No Degree",17.302679,15.668032,18.546383,15.093273
"Associate Degree,\\*Vocational",4.147304,4.458818,4.800157,4.817065
"Associate Degree,\\*Academic",3.735915,5.862863,4.392566,6.327216
Bachelor's Degree,17.179794,22.665527,20.353535,26.009243
Advanced Degree,8.692201,13.465665,10.572464,15.792258


In [13]:
tot = (df.groupby('EDUC').BASICWGT.sum() / 12000000).loc[['LTHS', 'HS', 'SC', 'COLL', 'ADV']]

In [14]:
tot.index = ['No High School Diploma', 'High School Diploma', 'Some College or Associate Degree', "Bachelor's Degree", 'Advanced Degree']

In [15]:
tot.to_csv(data_dir / 'cps_educ_tot.csv', index_label='name', header=True)

In [16]:
tot

No High School Diploma              21.497172
High School Diploma                 62.426027
Some College or Associate Degree    57.581616
Bachelor's Degree                   50.216704
Advanced Degree                     29.833912
Name: BASICWGT, dtype: float64

In [17]:
data

,2000,latest,2000_emp,latest_emp
No High School Diploma,16.080801,9.702842,10.038269,6.786475
High School Diploma,32.861298,28.176260,31.296618,25.174486
"Some College, No Degree",17.302679,15.668032,18.546383,15.093273
"Associate Degree,\\*Vocational",4.147304,4.458818,4.800157,4.817065
"Associate Degree,\\*Academic",3.735915,5.862863,4.392566,6.327216
Bachelor's Degree,17.179794,22.665527,20.353535,26.009243
Advanced Degree,8.692201,13.465665,10.572464,15.792258


In [18]:
ltdate = 'January 2020'

ba_adv_tot = tot["Bachelor's Degree"] + tot['Advanced Degree']
ba_adv_sh = data.loc["Bachelor's Degree", 'latest'] + data.loc['Advanced Degree', 'latest']

adv_tot = tot['Advanced Degree']
adv_sh = data.loc['Advanced Degree', 'latest']

sc_tot = tot['Some College or Associate Degree']

hs_tot = tot['High School Diploma']

lths_tot = tot['No High School Diploma']

text = (f'Over the year ending {ltdate}, {ba_adv_tot:.1f} million '+
        f'people over the age of 25, or {ba_adv_sh:.1f} percent of the total, '+
        f"have at least a bachelor's degree, with {adv_tot:.1f} million of "+
        f'those, or {adv_sh:.1f} percent of the total, holding '+
        "an advanced degree such as a master's degree, medical or law degree, or PhD. "+
        f'An additional {sc_tot:.1f} million people have some college coursework '+
        f'but no degree or have an associate degree. A total of {hs_tot:.1f} million '+
        f'have a high school diploma but no college, while {lths_tot:.1f} million '+
        f'have no high school diploma.')
write_txt(text_dir / 'cps_educ.txt', text)
write_txt(text_dir / 'cps_ltdate.txt', ltdate)

text

In [22]:
ba_adv_sh_pr = data.loc["Bachelor's Degree", '2000'] + data.loc['Advanced Degree', '2000']

ba_adv_sh_ch = ba_adv_sh - ba_adv_sh_pr

ba_adv_sh_emp = data.loc["Bachelor's Degree", 'latest_emp'] + data.loc['Advanced Degree', 'latest_emp']

ba_adv_sh_emp_pr = data.loc["Bachelor's Degree", '2000_emp'] + data.loc['Advanced Degree', '2000_emp']
ba_adv_sh_emp_ch = ba_adv_sh_emp - ba_adv_sh_emp_pr

In [23]:
text = ("The share of the population with a bachelor's degree or advanced degree "+
        f"increased by {ba_adv_sh_ch:.1f} percentage points since 2000. The increase "+
        "is even more pronounced among those who are employed; "+
        f"{ba_adv_sh_emp:.1f} percent have a college degree or advanced degree in {ltdate}, an "+
        f"increase of {ba_adv_sh_emp_ch:.1f} percentage points since 2000. ")

In [24]:
write_txt(text_dir / 'cps_educ2.txt', text)

In [25]:
write_txt(text_dir / 'cps_ltdate.txt', ltdate)